In [ ]:
# Clone and install the wandb-addons library
!git clone https://github.com/soumik12345/wandb-addons > clone.logs
!pip install ./wandb-addons[huggingface] > install.logs

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline

from wandb_addons.diffusers import StableDiffusionXLCallback
from wandb_addons.utils import autogenerate_seed

In [ ]:
# Define the Stable Diffusion XL pipeline
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)

# Set the device for the pipeline
pipeline = pipeline.to("cuda")

# Reduce memeory footprint
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
# Set the prompt
prompt = "A close up portrait of an old village woman, white hair, olive green eyes, dramatic light, dreamlike light, photorealism, ultra detail, 85mm lens, indian village backdrop, long exposure background"

# Set the negative prompt, or leave it `None` if you don't want to use it
negative_prompt = None

# Set the height and width of the generated image
height, witdth = 1024, 1024

# Automatically generate a seed for the generation, optionally give one of your own.
# This would ensure reproducibility of the experiment.
seed = autogenerate_seed()
generator = torch.Generator(device="cuda").manual_seed(seed)

# Additional configs for the experiment
configs = {
    "eta": 0.0,
    "guidance_rescale": 0.0,
    "seed": seed,
}

In [ ]:
# Create the WandB callback for StableDiffusionPipeline.
callback = StableDiffusionXLCallback(
    pipeline,
    prompt=prompt,
    negative_prompt=negative_prompt,
    wandb_project="diffusers-prompt-engineering",
    configs=configs,
)

# Add the callback to the pipeline, and execute the pipeline.
image = pipeline(
    prompt,
    negative_prompt=negative_prompt,
    generator=generator,
    height=height,
    width=witdth,
    callback=callback,
    eta=configs["eta"],
    guidance_rescale=configs["guidance_rescale"],
)